# Prediction of Ransome Activity / Construction of the features extractor

Rihem Mansri, Mohamed Issa, Mootez Dakhlaoui, Bourhan Dernayka, Joel Pascal Soffo, Ronny Tonato

TO DO LIST:
<br>
Wrap all of this in a class
<br>
Make different tests to choose the best metrics and baseline model
<br>
Remark 1 : Here, I did the standard scale on the whole dataset, which could not be a good pratice. A better thing to do would be to split the dataset, "transformer.fit_transform" the training set and "transformer.transform" the test set. 
<br>
Remark 2:
<br>
I did not encode the address column because 90% of the address are different, meaning there are only few repetitive addresses. Not sure it's necessary to encode(can be discussed). 

In [36]:
###importing the libraries
import os
import warnings
import pickle
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import boxcox
from tqdm.notebook import tqdm

from sklearn.model_selection import train_test_split
import pdb
import datetime
from scipy.stats import pearsonr

from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.metrics import accuracy_score, log_loss, plot_roc_curve, f1_score, precision_score, recall_score
from sklearn.model_selection import RandomizedSearchCV
import matplotlib
from prettytable import PrettyTable
from pprint import pprint

warnings.filterwarnings("ignore")

### Dataset
data = pd.read_csv("BitcoinHeistData.csv")

##converting days and years in a single columns(year/month/day)
from datetime import date, datetime

dat_temp = []
for i in range(data.shape[0]):
    aux = date.fromordinal(date(data['year'][i], 1, 1).toordinal() + data['day'][i]  - 1).strftime('%d/%m/%Y')
    dat_temp.append(aux)
data['time'] = dat_temp
data['time'] = pd.to_datetime(data['time'])

###Converting labels to binary
data.loc[data['label'] != 'white', 'label'] = 1
data.loc[data['label'] == 'white', 'label'] = 0
Y = data.pop('label')

###Deleting day and year columns(became useless)
data.pop('day')
data.pop('year')

###Transformation of the data
cnt_featnames = ['length', 'weight', 'count', 'looped', 'neighbors', 'income']
date_featnames = ['time']

from sklearn.base import TransformerMixin, BaseEstimator
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from pandas import Timestamp

# First we create a scikit-learn encoder that computes 
#  the age in days of columns containing dates
class AgeEncoder(TransformerMixin, BaseEstimator):
    def fit(self, X, y=None):
        self.today = Timestamp.today()
        return self

    def transform(self, X):
        return X.apply(lambda x: (x - self.today).dt.days, axis=0)

# Centers and reduces (variance=1) columns
standard_scaler = StandardScaler()

# One-hot encode, similar to pd.get_dummies
# one_hot_encoder = OneHotEncoder(drop='first', sparse=False)

# A pipeline that first computes age, and standardizes it
scaled_age_encoder = Pipeline([
    ('age', AgeEncoder()),
    ('scaling', StandardScaler())
])

# Let's combine all these transformations
transformer = ColumnTransformer([
    ('standard_scaling', standard_scaler, cnt_featnames),
    ('dates_age_scaled', scaled_age_encoder, date_featnames)
])

data_prep = transformer.fit_transform(data)

##turn the neww array into dataframe
data_prep = pd.DataFrame(data_prep, columns=['length', 'weight', 'count', 'looped', 'neighbors', 'income', 'time'])
data_prep['label'] = Y

##save as pickle
data_prep.to_pickle("data_prep_df.pkl")